In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
import logging
from collections import Counter
import datetime
import os
import yaml
from sentinelhub import DataSource
import scipy.sparse as sparse
import scipy
from scipy.sparse.linalg import splu
from skimage.transform import resize
from sentinelhub import CustomUrlParam
from time import time as timer
from time import sleep as sleep
import multiprocessing
import math
import reverse_geocoder as rg
import pycountry
import pycountry_convert as pc
import hickle as hkl


/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
%run ../src/utils/slope.py
%run ../src/utils/utils-bilinear.py
%run ../src/dsen2/utils/DSen2Net.py

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [3]:
dates = ('2018-12-15', '2020-01-15')
SIZE = 9*5
IMSIZE = (SIZE * 14)+2

cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

In [4]:
landscapes = {
    'ethiopia-tigray': (13.540810, 38.177220),
    'kenya-makueni-2': (-1.817109, 37.44563),
    'ghana': (9.259359, -0.83375),
    'niger-koure': (13.18158, 2.478),
    'cameroon-farnorth': (10.596, 14.2722),
    'mexico-campeche': (18.232495, -92.1234215),
    'malawi-rumphi': (-11.044, 33.818),
    'ghana-sisala-east': (10.385, -1.765),
    'ghana-west-mamprusi': (10.390084, -0.846330),
    'ghana-kwahu': (6.518909, -0.826008),
    'senegal-16b': (15.82585, -15.34166),
    'india-kochi': (9.909, 76.254),
    'india-sidhi': (24.0705, 81.607),
    'brazil-esperito-santo': (-20.147, -40.837),
    'brazil-paraiba': (-22.559943, -44.186629),
    'brazil-goias': (-14.905595, -48.907399),
    'colombia-talima': (4.179529, -74.889171),
    'drc-kafubu': (-11.749636, 27.586622),
    'thailand-khon-kaen': (15.709725, 102.546518),
    'indonesia-west-java': (-6.721101, 108.280949),
    'madagascar': (-18.960152, 47.469587),
    'tanzania': (-6.272258, 36.679824),
    'chile': (-36.431237, -71.872030),
    'indonesia-jakarta': (-6.352580, 106.677072),
    'caf-baboua': (5.765917, 14.791618),   
    'honduras': (14.096664, -88.720304),
    'nicaragua': (12.398014, -86.963042),
    'china': (26.673679, 107.464231),
    'australia-west': (-32.666762, 117.411197),
    'mexico-sonora': (29.244288, -111.243230),
    'south-africa': (-30.981698, 28.727301),
    'maldonado-uraguay': (-34.629250, -55.004331),
    'dominican-rep-la-salvia': (18.872589, -70.462961)
}

landscape = 'dominican-rep-la-salvia'

#coords = (7.702058, -0.709011) # brong ahafo, bono east
#coords = (7.398111, -1.269223) # cocoa
#coords = (16.032170, -90.144511) # Guatemala
#coords = (13.757749, -90.004949) # elsalvador imposible
#coords = (13.727334, -90.015579) # elsalvador imposible2
#coords = (13.933745, -84.690842) # Bonanza, Nicaragua

OUTPUT_FOLDER = '../tile_data/{}/'.format(landscape)
coords = landscapes[landscape]
coords = (coords[1], coords[0])
print(OUTPUT_FOLDER, coords)

../tile_data/dominican-rep-la-salvia/ (-70.462961, 18.872589)


In [5]:
landscape_df = pd.DataFrame({'landscape': [x for x in landscapes.keys()], 
                             'latitude': [x[0] for x in landscapes.values()],
                             'longitude': [x[1] for x in landscapes.values()]
})

landscape_df.to_csv("../data/latlongs/landscapes.csv", index=False)
print(len(landscape_df))

33


# Helper functions (to be moved to a utils file)

In [6]:
EPSG = CRS.WGS84
GRID_SIZE_X = 1
GRID_SIZE_Y = 1

IMAGE_X = 14*GRID_SIZE_X
IMAGE_Y = 14*GRID_SIZE_Y

TEST_X = 5
TEST_Y = 5

with open("../config.yaml", 'r') as stream:
    key = (yaml.safe_load(stream))
    API_KEY = key['key']

In [7]:
# These arrays are for smoothly overlapping the cloud and shadow interpolation
c_arr = np.array([[1, 1, 1, 1, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 2, 3, 2, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 1, 1, 1, 1,],])
                  
c_arr = c_arr / 3
o_arr = 1 - c_arr
c_arr = np.tile(c_arr[:, :, np.newaxis], (1, 1, 10))
o_arr = np.tile(o_arr[:, :, np.newaxis], (1, 1, 10))

def convertCoords(xy, src='', targ=''):

    srcproj = osr.SpatialReference()
    srcproj.ImportFromEPSG(src)
    targproj = osr.SpatialReference()
    if isinstance(targ, str):
        targproj.ImportFromProj4(targ)
    else:
        targproj.ImportFromEPSG(targ)
    transform = osr.CoordinateTransformation(srcproj, targproj)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(xy[0], xy[1])
    pt.Transform(transform)
    return([pt.GetX(), pt.GetY()])

In [8]:
def calculate_epsg(points):
    lon, lat = points[0], points[1]
    print(lon, lat)
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return int(epsg_code)

def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area)
    return area
    
def offset_x(coord, offset):
    ''' Converts a WGS 84 to UTM, adds meters, and converts back'''
    epsg = calculate_epsg(coord)
    coord = convertCoords(coord, 4326, epsg)
    coord[0] += offset
    coord = convertCoords(coord, epsg, 4326)
    return coord
    
def offset_y(coord, offset):
    ''' Converts a WGS 84 to UTM, adds meters, and converts back'''
    epsg = calculate_epsg(coord)
    coord = convertCoords(coord, 4326, epsg)
    coord[1] += offset
    coord = convertCoords(coord, epsg, 4326)
    return coord

def calculate_area(bbx):
    '''
    Calculates the area in ha of a [(min_x, min_y), (max_x, max_y)] bbx
    '''

    epsg = calculate_epsg(bbx[0])
    
    mins = convertCoords(bbx[0], 4326, epsg)
    maxs = convertCoords(bbx[1], 4326, epsg)
    area = PolygonArea([(mins[0], mins[1]), # BL
                        (mins[0], maxs[1]), # BR
                        (maxs[0], mins[1]), # TL
                        (maxs[0], mins[1]) # TR
                        ])
    hectares = math.floor(area / 1e4)
    print(hectares)
    

def calculate_bbx(coord, step_x, step_y, expansion, multiplier = 1.):
    ''' Calculates the four corners of a bounding box of step_x * step_y offset from coord'''
    coord_bl = np.copy(coord)
    coord1 = offset_x(coord_bl, 6300*step_x - expansion)
    coord1 = offset_y(coord1 , 6300*step_y - expansion)
    
    coord_tr = np.copy(coord)
    coord2 = offset_x(coord_tr, 6300*(step_x + multiplier) + expansion)
    coord2 = offset_y(coord2, 6300*(step_y + multiplier) + expansion)
    bbx = (coord2, coord1)
    return bbx

# Data download

In [9]:
def try_function(func, n_tries = 5, *args, **kwargs):
    for try_ in range(0, n_tries):
        try:
            returns = func(*args, **kwargs)
        except Exception:
            print("Trying again in 20 seconds")
            sleep(20)
        else:
            continue

In [10]:
def identify_clouds(bbox, epsg = EPSG, dates = dates):
    '''
    - Download the CLOUD_DETECTION layer from sentinel-hub
    - Process cloud probabiiliity
    - Return cloud probability map
    '''
    for try_ in range(0, 5):
        try:
            box = BBox(bbox, crs = epsg)
            cloud_request = WmsRequest(
                layer='CLOUD_DETECTION',
                bbox=box,
                time=dates,
                width=(5*9*14)+2,
                height=(5*9*14)+2,
                image_format = MimeType.TIFF_d16,
                maxcc=0.7,
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=48),
            )

            cloud_img = cloud_request.get_data()
            cloud_img = np.array(cloud_img) / 65535
            cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(cloud_img))
            shadows = mcm_shadow_mask(np.array(cloud_img), cloud_probs)
            print("Cloud probs: {}".format(cloud_probs.shape))
            return cloud_img, cloud_probs, shadows
        except Exception as e:
            #logging.fatal(e, exc_info=True)
            sleep(30)
    
    
def download_dem(bbox, epsg = EPSG):
    #@valid
    for try_ in range(0, 5):
        try:
            box = BBox(bbox, crs = epsg)
            dem_s = (630)+4
            dem_request = WmsRequest(data_source=DataSource.DEM,
                                 layer='DEM',
                                 bbox=box,
                                 width=dem_s,
                                 height=dem_s,
                                 instance_id=API_KEY,
                                 image_format=MimeType.TIFF_d32f,
                                 custom_url_params={CustomUrlParam.SHOWLOGO: False})
            dem_image = dem_request.get_data()[0]
            dem_image = calcSlope(dem_image.reshape((1, dem_s, dem_s)),
                          np.full((dem_s, dem_s), 10), np.full((dem_s, dem_s), 10), zScale = 1, minSlope = 0.02)
            dem_image = dem_image.reshape((dem_s,dem_s, 1))
            dem_image = dem_image[1:dem_s-1, 1:dem_s-1, :]
            return dem_image #/ np.max(dem_image)
        except Exception as e:
            #logging.fatal(e, exc_info=True)
            sleep((try_+1)*30)
            pass

def download_layer(bbox, epsg = EPSG, dates = dates, year = 2019):
    #for try_ in range(5):
        #try:
    box = BBox(bbox, crs = epsg)
    image_request = WcsRequest(
            layer='L2A20',
            bbox=box,
            time=dates,
            image_format = MimeType.TIFF_d16,
            maxcc=0.7,
            resx='10m', resy='10m',
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=48),
        )
    img_bands = image_request.get_data()
    img_20 = np.stack(img_bands)
    print("Original size: {}".format(img_20.shape))
    img_20 = resize(img_20, (img_20.shape[0], 632, 632, img_20.shape[-1]), order = 0)

    image_request = WcsRequest(
            layer='L2A10',
            bbox=box,
            time=dates,
            image_format = MimeType.TIFF_d32f,
            maxcc=0.7,
            resx='10m', resy='10m',
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'BICUBIC',
                                constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
            time_difference=datetime.timedelta(hours=48),
    )

    img_bands = image_request.get_data()
    img_10 = np.stack(img_bands)
    print("Original 10 size: {}".format(img_10.shape))
    img_10 = resize(img_10, (img_10.shape[0], 632, 632, img_10.shape[-1]), order = 0)

    img = np.concatenate([img_10, img_20], axis = -1)

    image_dates = []
    for date in image_request.get_dates():
        if date.year == year - 1:
            image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == year:
            image_dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == year + 1:
            image_dates.append(365 + starting_days[(date.month-1)]+date.day)
    image_dates = np.array(image_dates)

    return img, image_dates#, shadows, shadow_steps

        #except Exception as e:
            #logging.fatal(e, exc_info=True)
            #sleep((try_+1)*30)
        
        
        
def download_sentinel_1(bbox, epsg = EPSG, imsize = 632, 
                        dates = dates, layer = "SENT", year = 2019):
    #for try_ in range(5):
        #try:
    box = BBox(bbox, crs = epsg)
    image_request = WcsRequest(
            layer=layer,
            bbox=box,
            time=dates,
            image_format = MimeType.TIFF_d16,
            maxcc=1.0,
            resx='5m', resy='5m',
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=48),
        )
    img_bands = image_request.get_data()
    s1 = np.stack(img_bands)
    s1 = resize(s1, (s1.shape[0], imsize*2, imsize*2, s1.shape[-1]), order = 0)
    s1 = np.reshape(s1, (s1.shape[0], s1.shape[1]//2, 2, s1.shape[2] // 2, 2, s1.shape[-1]))
    s1 = np.mean(s1, (2, 4))
    #s1 = s1[:, 8:24, 8:24, :]

    image_dates = []
    for date in image_request.get_dates():
        if date.year == year - 1:
            image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == year:
            image_dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == year + 1:
            image_dates.append(365 + starting_days[(date.month-1)]+date.day)
    image_dates = np.array(image_dates)
    s1c = np.copy(s1)
    s1c[np.where(s1c < 1.)] = 0
    n_pix_oob = np.sum(s1c, axis = (1, 2, 3))
    print("OOB", n_pix_oob)
    #to_remove = np.argwhere(np.max(s1, (1, 2, 3)) == 1.).flatten()
    to_remove = np.argwhere(n_pix_oob > (imsize*2*imsize*2)/50)
    s1 = np.delete(s1, to_remove, 0)
    print(np.max(s1, (1, 2, 3)))
    image_dates = np.delete(image_dates, to_remove)
    return s1, image_dates

        #except Exception as e:
            #logging.fatal(e, exc_info=True)
            #sleep((try_+1)*30)

def identify_s1_layer(coords):
    results = rg.search(coords)
    country = results[-1]['cc']
    continent_name = pc.country_alpha2_to_continent_code(country)
    if continent_name in ['AF', 'OC']:
        layer = "SENT"
    if continent_name in ['SA']:
        if coords[0] > -7.11:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['AS']:
        if coords[0] > 23.3:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['NA']:
        layer = "SENT_DESC"
    print(continent_name)
    print(layer)
    return layer

# Cloud and shadow removal

In [11]:
def remove_cloud_and_shadows(tiles, c_probs, shadows, image_dates):
    '''
    - Iterate through a window size over each time step
    - Identify whether or not there are clouds and shadows there
    - If there are, interpolate the 5*5 window with clean imagery for all bands
    '''
    wsize = 5
    c_probs = c_probs - np.min(c_probs, axis = 0)
    c_probs[np.where(c_probs > 0.33)] = 1.
    c_probs[np.where(c_probs < 0.33)] = 0.
    c_probs = np.reshape(c_probs, (c_probs.shape[0], 632//8, 8, 632//8, 8))
    c_probs = np.sum(c_probs, (2, 4))
    c_probs = resize(c_probs, (c_probs.shape[0], 632, 632), 0)
    c_probs[np.where(c_probs < 16)] = 0
    c_probs[np.where(c_probs >= 16)] = 1
    secondary_c_probs = np.copy(c_probs)
    c_probs += shadows
    c_probs[np.where(c_probs >= 1.)] = 1.
    number_interpolated = 0
    for cval in tnrange(0, IMSIZE - 4, 2):
        for rval in range(0, IMSIZE - 4, 2):
            subs = c_probs[:, cval:cval + wsize, rval:rval+wsize]
            sums = np.sum(subs, axis = (1, 2))
            satisfactory = [x for x in range(c_probs.shape[0]) if sums[x] < 8]
            satisfactory = np.array(satisfactory)
            for date in range(0, tiles.shape[0]):
                if np.sum(subs[date, :, :]) > 8:
                    number_interpolated += 1
                    before, after = calculate_proximal_steps(date, satisfactory)
                    before = date + before
                    after = date + after
                    bef = tiles[before, cval:cval+wsize, rval:rval+wsize, : ]
                    aft = tiles[after, cval:cval+wsize, rval:rval+wsize, : ]
                    before = image_dates[before]
                    after = image_dates[after]
                    before_diff = abs(image_dates[date] - before)
                    after_diff = abs(image_dates[date] - after)
                    bef_wt = 1 - before_diff / (before_diff + after_diff)
                    aft_wt = 1 - bef_wt
                    candidate = bef_wt*bef + aft_wt*aft
                    candidate = candidate*c_arr + tiles[date, cval:cval+wsize, rval:rval+wsize, : ]*o_arr
                    tiles[date, cval:cval+wsize, rval:rval+wsize, : ] = candidate 
    print("A total of {} pixels were interpolated".format(number_interpolated))
    return tiles, c_probs, secondary_c_probs

def remove_missed_clouds(img):
    iqr = np.percentile(img[:, :, :, 3].flatten(), 75) - np.percentile(img[:, :, :, 3].flatten(), 25)
    thresh_t = np.percentile(img[:, :, :, 3].flatten(), 75) + iqr*2
    thresh_b = np.percentile(img[:, :, :, 3].flatten(), 25) - iqr*2
    diffs_fw = np.diff(img, 1, axis = 0)
    diffs_fw = np.mean(diffs_fw, axis = (1, 2, 3))
    diffs_fw = np.array([0] + list(diffs_fw))
    diffs_bw = np.diff(np.flip(img, 0), 1, axis = 0)
    diffs_bw = np.flip(np.mean(diffs_bw, axis = (1, 2, 3)))
    diffs_bw = np.array(list(diffs_bw) + [0])
    diffs = abs(diffs_fw - diffs_bw) * 100 # 3, -3 -> 6, -3, 3 -> 6, -3, -3
    #diffs = [int(x) for x in diffs]
    outlier_percs = []
    for step in range(img.shape[0]):
        bottom = len(np.argwhere(img[step, :, :, 3].flatten() > thresh_t))
        top = len(np.argwhere(img[step, :, :, 3].flatten() < thresh_b))
        p = 100* ((bottom + top) / (IMSIZE*IMSIZE))
        outlier_percs.append(p)
    to_remove = np.argwhere(np.array(outlier_percs) > 20)
    return to_remove

# Data interpolation

In [12]:
diagonals = np.zeros(2*2+1)
diagonals[2] = 1.
for i in range(2):
    diff = diagonals[:-1] - diagonals[1:]
    diagonals = diff
offsets = np.arange(2+1)
shape = (70, 72)

def smooth(y, lmbd = 800, diagonals = diagonals, offsets = offsets, shape = shape, d = 2):
    ''' 
    Apply whittaker smoothing to a 1-dimensional array, returning a 1-dimensional array
    '''
    E = sparse.eye(72, format = 'csc')
    D = scipy.sparse.diags(diagonals, offsets, shape)
    coefmat = E + lmbd * D.conj().T.dot(D)
    z = splu(coefmat).solve(y)
    return z

def calculate_and_save_best_images(img_bands, image_dates):
    '''
    
    '''

    biweekly_dates = [day for day in range(0, 360, 5)] # ideal imagery dates are every 15 days
    
    # Identify the dates where there is < 20% cloud cover
    #satisfactory_ids = list(np.argwhere(np.array(means) < 4.).reshape(-1, )) 
    satisfactory_ids = [x for x in range(0, img_bands.shape[0])]
    satisfactory_dates = [value for idx, value in enumerate(image_dates) if idx in satisfactory_ids]
    
    
    selected_images = {}
    for i in biweekly_dates:
        distances = [abs(date - i) for date in satisfactory_dates]
        closest = np.min(distances)
        closest_id = np.argmin(distances)
        # If there is imagery within 8 days, select it
        if closest < 8:
            date = satisfactory_dates[closest_id]
            image_idx = int(np.argwhere(np.array(image_dates) == date)[0])
            selected_images[i] = {'image_date': [date], 'image_ratio': [1], 'image_idx': [image_idx]}
        # If there is not imagery within 8 days, look for the closest above and below imagery
        else:
            distances = np.array([(date - i) for date in satisfactory_dates])
            # Number of days above and below the selected date of the nearest clean imagery
            above = distances[np.where(distances < 0, distances, -np.inf).argmax()]
            below = distances[np.where(distances > 0, distances, np.inf).argmin()]
            if abs(above) > 240: # If date is the last date, occassionally argmax would set above to - number
                above = below
            if abs(below) > 240:
                below = above
            if above != below:
                below_ratio = above / (above - below)
                above_ratio = 1 - below_ratio
            else:
                above_ratio = below_ratio = 0.5
                
            # Extract the image date and imagery index for the above and below values
            above_date = i + above
            above_image_idx = int(np.argwhere(np.array(image_dates) == above_date)[0])
            
            below_date = i + below
            below_image_idx = int(np.argwhere(np.array(image_dates) == below_date)[0])
            
            selected_images[i] = {'image_date': [above_date, below_date], 'image_ratio': [above_ratio, below_ratio],
                                 'image_idx': [above_image_idx, below_image_idx]}
                            
    max_distance = 0
    
    for i in selected_images.keys():
        #print(i, selected_images[i])
        if len(selected_images[i]['image_date']) == 2:
            dist = selected_images[i]['image_date'][1] - selected_images[i]['image_date'][0]
            if dist > max_distance:
                max_distance = dist
    
    print("Maximum time distance: {}".format(max_distance))
        
    keep_steps = []
    for i in selected_images.keys():
        info = selected_images[i]
        if len(info['image_idx']) == 1:
            step = img_bands[info['image_idx'][0]]
        if len(info['image_idx']) == 2:
            step1 = img_bands[info['image_idx'][0]] * 0.5#info['image_ratio'][0]
            step2 = img_bands[info['image_idx'][1]] * 0.5 #info['image_ratio'][1]
            step = step1 + step2
        keep_steps.append(step)
        
    keep_steps = np.stack(keep_steps)
    return keep_steps



# Tiling and coordinate selection functions

In [13]:
MDL_PATH = "../src/dsen2/models/"

input_shape = ((4, None, None), (6, None, None))
model = s2model(input_shape, num_layers=6, feature_size=128)
predict_file = MDL_PATH+'s2_032_lr_1e-04.hdf5'
print('Symbolic Model Created.')

model.load_weights(predict_file)

def DSen2(d10, d20):
    test = [d10, d20]
    input_shape = ((4, None, None), (6, None, None))
    prediction = _predict(test, input_shape, deep=False)
    #prediction *= 5
    return prediction

def _predict(test, input_shape, model = model, deep=False, run_60=False):
    
    print("Predicting using file: {}".format(predict_file))
    prediction = model.predict(test, verbose=1)
    return prediction

Symbolic Model Created.


In [14]:
days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
starting_days = np.cumsum(days_per_month)

In [15]:
def calculate_proximal_steps(date, satisfactory):
    arg_before = None
    arg_after = None
    if date > 0:
        idx_before = satisfactory - date
        arg_before = idx_before[np.where(idx_before < 0, idx_before, -np.inf).argmax()]
    if date < np.max(satisfactory):
        idx_after = satisfactory - date
        arg_after = idx_after[np.where(idx_after > 0, idx_after, np.inf).argmin()]
    if not arg_after and not arg_before:
        arg_after = date
        arg_before = date
    if not arg_after:
        arg_after = arg_before
    if not arg_before:
        arg_before = arg_after
    #print(arg_before, date, arg_after)
    return arg_before, arg_after

In [16]:
def rank_array(arr):
    order = arr.argsort()
    ranks = order.argsort()
    return ranks

In [17]:
def mcm_shadow_mask(arr, c_probs):
    #' From "Cloud and cloud shadow masking for Sentinel-2 using multi-
    #         temporal images in global area. Candra et al. 2020."
    #'[B02,B03,B04, B08, B05,B06,B07, B8A,B11,B12]
    #'[ 0 , 1 , 2 , 3  , 4  , 5,  6   7  , 8  , 9]'
    mean_c_probs = np.mean(c_probs, axis = (1, 2))
    cloudy_steps = np.argwhere(mean_c_probs > 0.25)
    images_clean = np.delete(arr, cloudy_steps, 0)
    cloud_ranks = rank_array(mean_c_probs)
    diffs = abs(np.sum(arr - np.mean(images_clean, axis = 0), axis = (1, 2, 3)))
    diff_ranks = rank_array(diffs)
    overall_rank = diff_ranks + cloud_ranks
    reference_idx = np.argmin(overall_rank)
    ri = arr[reference_idx]
    print(reference_idx)
    
    nir_means = np.mean(arr[:, :, :, 4], axis = (0))
    
    shadows = np.zeros((arr.shape[0], 632, 632))    
    # Candra et al. 2020
    
    for time in tnrange(arr.shape[0]):
        for x in range(arr.shape[1]):
            for y in range(arr.shape[2]):
                ti_slice = arr[time, x, y]
                ri_slice = ri[x, y]
                deltab2 = ti_slice[1] - ri_slice[1]
                deltab3 = ti_slice[2] - ri_slice[2]
                deltab4 = ti_slice[3] - ri_slice[3]
                deltab8a = ti_slice[6] - ri_slice[6]
                deltab11 = ti_slice[8] - ri_slice[8]

                if deltab2 < 0.1:
                    if deltab3 < 0.08:
                        if deltab4 < 0.08:
                            if deltab8a < -0.04:
                                if deltab11 < -0.04:
                                    if ti_slice[1] < 0.0950:
                                        shadows[time, x, y] = 1.
                                                       
                            
    # Remove shadows if cannot coreference a cloud
    shadow_large = np.reshape(shadows, (shadows.shape[0], 79, 8, 79, 8))
    shadow_large = np.sum(shadow_large, axis = (2, 4))
    
    cloud_large = np.copy(c_probs)
    cloud_large[np.where(c_probs > 0.33)] = 1.
    cloud_large[np.where(c_probs < 0.33)] = 0.
    cloud_large = np.reshape(cloud_large, (shadows.shape[0], 79, 8, 79, 8))
    cloud_large = np.sum(cloud_large, axis = (2, 4))
    for time in tnrange(shadow_large.shape[0]):
        for x in range(shadow_large.shape[1]):
            x_low = np.max([x - 8, 0])
            x_high = np.min([x + 8, shadow_large.shape[1] - 1])
            for y in range(shadow_large.shape[2]):
                y_low = np.max([y - 8, 0])
                y_high = np.min([y + 8, shadow_large.shape[1] - 1])
                if shadow_large[time, x, y] < 8:
                    shadow_large[time, x, y] = 0.
                if shadow_large[time, x, y] >= 8:
                    shadow_large[time, x, y] = 1.
                c_prob_window = cloud_large[time, x_low:x_high, y_low:y_high]
                if np.max(c_prob_window) < 16:
                    shadow_large[time, x, y] = 0.
                    
    shadow_large = resize(shadow_large, (shadow_large.shape[0], 632, 632), order = 0)
    shadows *= shadow_large
    
    # Go through and aggregate the shadow map to an 80m grid, and extend it one grid size around
    # any positive ID
    
    shadows = np.reshape(shadows, (shadows.shape[0], 79, 8, 79, 8))
    shadows = np.sum(shadows, axis = (2, 4))
    shadows[np.where(shadows < 12)] = 0.
    shadows[np.where(shadows >= 12)] = 1.
    
    shadows = resize(shadows, (shadows.shape[0], 632, 632), order = 0)
    shadows = np.reshape(shadows, (shadows.shape[0], 632//4, 4, 632//4, 4))
    shadows = np.max(shadows, (2, 4))
    
    shadows_new = np.zeros_like(shadows)
    for time in range(shadows.shape[0]):
        for x in range(shadows.shape[1]):
            for y in range(shadows.shape[2]):
                if shadows[time, x, y] == 1:
                    min_x = np.max([x - 1, 0])
                    max_x = np.min([x + 2, 157])
                    min_y = np.max([y - 1, 0])
                    max_y = np.min([y + 2, 157])
                    for x_idx in range(min_x, max_x):
                        for y_idx in range(min_y, max_y):
                            shadows_new[time, x_idx, y_idx] = 1.
    shadows_new = resize(shadows_new, (shadows.shape[0], 632, 632), order = 0)
    return shadows_new

In [18]:
def find_and_make_dirs(dirs):
    if not os.path.exists(os.path.realpath(dirs)):
        os.makedirs(os.path.realpath(dirs))

def make_output_and_temp_folders(idx, output_folder = OUTPUT_FOLDER):
    find_and_make_dirs(output_folder + "raw/")
    find_and_make_dirs(output_folder + "raw/clouds/")
    find_and_make_dirs(output_folder + "raw/s1/")
    find_and_make_dirs(output_folder + "raw/s2/")
    find_and_make_dirs(output_folder + "raw/misc/")
    find_and_make_dirs(output_folder + "processed/")

def download_large_tile(coord, step_x, step_y, folder = OUTPUT_FOLDER, year = 2019, s1_layer = "SENT"):
    '''
    Ideally the folder structure should be as follows:
     - Raw
         - Cloud
              - Y*5, X*5
         - Sentinel 1
         - Sentinel 2
         - DEM
         - Dates
     - Processed
         - Y, X
         
    Methodological questions to address:
        - Should we download all images for a region first, and then process?     - Seems unnecessary
          This would allow for matching up the borders
        - Remove dirty steps for the whole tile or by subtiles 
        - Is this the proper time to address the interpolation of clouds (?)
        - Make sure that cloud shadows line up with clouds                        - X
        - Make sure that sentinel 1 is properly matched up and items are removed  - X
         
    The strategy is as follows:
        - Download 5*9 by 5*9 cloud, S1, S2, DEM, dates, save to folders          - X
        - Load each start:start+128 tile                                          - X
        - Remove dirty steps, missing images, missed clouds, ratios               - X
        - Superresolve                                                            - X
        - Indices                                                                 - X
        - Calculate_and_save_best_images                                          - X
        - Whittaker smooth                                                        - X
        - Fuse S1
        - Save to processed / y / x
        - Iterate for each of the large sub-tiles
    
    '''
    print(coord)
    bbx = calculate_bbx(coord, step_x, step_y, expansion = 10)
    print(bbx)
    dem_bbx = calculate_bbx(coord, step_x, step_y, expansion = 20)
    idx = str(step_y) + "_" + str(step_x)
    print(idx)
    idx = str(idx)
    #if 1 != 0:
    print("Making raw folders")
    make_output_and_temp_folders(idx)

    print("Calculating cloud cover")
    if not os.path.exists(folder + "raw/clouds/clouds_{}.npy".format(idx)):
        l1c, cloud_probs, shadows = identify_clouds(bbx) # integrate cloud shadow here
        hkl.dump(cloud_probs, "raw/clouds/clouds_{}.npy".format(idx), mode='w', compression='gzip')
        hkl.dump(shadows, "raw/clouds/shadows_{}.npy".format(idx), mode='w', compression='gzip')

    if not os.path.exists(folder + "raw/s1/{}.npy".format(idx)):
        print("Downloading S1")
        s1_layer = identify_s1_layer((coord[1], coord[0]))
        s1, s1_dates = download_sentinel_1(bbx, layer = s1_layer)
        s1 = process_sentinel_1_tile(s1, s1_dates)
        hkl.dump(s1, "raw/s1/{}.npy".format(idx), mode='w', compression='gzip')
        hkl.dump(s1_dates, "raw/misc/s1_dates{}.npy".format(idx), mode='w', compression='gzip')

    if not os.path.exists(folder + "raw/s2/{}.npy".format(idx)):
        print("Downloading S2")
        s2, s2_dates = download_layer(bbx)
        hkl.dump(s2, "raw/s2/{}.npy".format(idx), mode='w', compression='gzip')
        hkl.dump(s2_dates, "raw/misc/s2_dates_{}.npy".format(idx), mode='w', compression='gzip')

    if not os.path.exists(folder + "raw/misc/dem_{}.npy".format(idx)):
        print("Downloading DEM")
        dem = download_dem(dem_bbx) # get the DEM BBOX
        hkl.dump(dem, "raw/misc/dem_{}.npy".format(idx), mode='w', compression='gzip')

In [19]:
def calculate_bad_steps(sentinel2, clouds):
    n_cloud_px = np.array([len(np.argwhere(clouds[x, :, :].reshape((632)*(632)) > 0.3)) for x in range(clouds.shape[0])])
    cloud_steps = np.argwhere(n_cloud_px > 632**2 / 5)
    missing_images = [np.argwhere(sentinel2[x, :, : :].flatten() == 0.0) for x in range(sentinel2.shape[0])]
    missing_images = np.array([len(x) for x in missing_images])
    missing_images_p = [np.argwhere(sentinel2[x, :, : :10].flatten() >= 1) for x in range(sentinel2.shape[0])]
    missing_images_p = np.array([len(x) for x in missing_images_p])
    missing_images += missing_images_p
    missing_images = np.argwhere(missing_images >= 100)
    print(cloud_steps)
    print(missing_images)
    #perc_shadow_px = np.sum(shadows, axis = (1, 2) / (79**2))
    #shadow_steps = np.argwhere(perc_shadow_px > 20)
    to_remove = np.unique(np.concatenate([cloud_steps.flatten(), missing_images.flatten()]))#, shadow_steps]))
    return to_remove

def superresolve(sentinel2):
    d10 = sentinel2[:, :, :, 0:4]
    d20 = sentinel2[:, :, :, 4:10]

    d10 = np.swapaxes(d10, 1, -1)
    d10 = np.swapaxes(d10, 2, 3)
    d20 = np.swapaxes(d20, 1, -1)
    d20 = np.swapaxes(d20, 2, 3)
    superresolved = DSen2(d10, d20)
    superresolved = np.swapaxes(superresolved, 1, -1)
    superresolved = np.swapaxes(superresolved, 1, 2)

    # returns band IDXs 3, 4, 5, 7, 8, 9
    return superresolved

def process_sentinel_1_tile(sentinel1, dates):
    s1 = calculate_and_save_best_images(sentinel1, dates)
    # Retain only iamgery every 15 days
    biweekly_dates = np.array([day for day in range(0, 360, 5)])
    to_remove = np.argwhere(biweekly_dates % 15 != 0)
    s1 = np.delete(s1, to_remove, 0)
    return s1


def interpolate_array(x):
    no_dem = np.delete(x, 10, -1)
    no_dem = np.reshape(no_dem, (72, 128*128*14))
    no_dem = np.swapaxes(no_dem, 0, 1)

    pool = multiprocessing.Pool(6)
    no_dem = pool.map(smooth, no_dem)
    pool.close()
    pool.join()

    no_dem = np.swapaxes(no_dem, 0, 1)
    no_dem = np.reshape(no_dem, (72, 128, 128, 14))
    x[:, :, :, :10] = no_dem[:, :, :, :10]
    x[:, :, :, 11:] = no_dem[:, :, :, 10:]

    biweekly_dates = np.array([day for day in range(0, 360, 5)])
    to_remove = np.argwhere(biweekly_dates % 15 != 0)
    x = np.delete(x, to_remove, 0)
    return x

def process_large_tile(step_x, step_y, folder = OUTPUT_FOLDER):
    idx = str(step_y) + "_" + str(step_x)
    x_vals = []
    y_vals = []
    # save to disk
    for i in range(25):
        y_val = (24 - i) // 5
        x_val = 5 - ((25 - i) % 5)
        x_val = 0 if x_val == 5 else x_val
        x_vals.append(x_val)
        y_vals.append(y_val)
        
    y_vals = [i + (5*step_y) for i in y_vals]
    x_vals = [i + (5*step_x) for i in x_vals]
    print(y_vals, x_vals)
    
    
    processed = True
    for x, y in zip(x_vals, y_vals):
        if not os.path.exists(folder + "processed/{}/{}.npy".format(str(y), str(x))):
            processed = False
    if not processed:
        
        clouds = hkl.load(folder + "raw/clouds/clouds_{}.npy".format(idx))
        sentinel1 = hkl.load(folder + "raw/s1/{}.npy".format(idx))
        radar_dates = hkl.load(folder + "raw/misc/s1_dates_{}.npy".format(idx))
        sentinel2 = hkl.load(folder + "raw/s2/{}.npy".format(idx))
        dem = hkl.load(folder + "raw/misc/dem_{}.npy".format(idx))
        image_dates = hkl.load(folder + "raw/misc/s2_dates_{}.npy".format(idx))
        if os.path.exists(folder + "raw/clouds/shadows_{}.npy".format(idx)):
            shadows = hkl.load(folder + "raw/clouds/shadows_{}.npy".format(idx))
        else:
            print("No shadows file, so calculating shadows with L2A")
            shadows = mcm_shadow_mask(sentinel2, clouds)
        print("The files have been loaded")

        #sentinel1 = process_sentinel_1_tile(sentinel1, radar_dates)
        to_remove = calculate_bad_steps(sentinel2, clouds)
        sentinel2 = np.delete(sentinel2, to_remove, axis = 0)
        clouds = np.delete(clouds, to_remove, axis = 0)
        shadows = np.delete(shadows, to_remove, axis = 0)
        image_dates = np.delete(image_dates, to_remove)
        print("Cloudy and missing images removed, radar processed")

        to_remove = remove_missed_clouds(sentinel2)
        sentinel2 = np.delete(sentinel2, to_remove, axis = 0)
        clouds = np.delete(clouds, to_remove, axis = 0)
        image_dates = np.delete(image_dates, to_remove)
        shadows = np.delete(shadows, to_remove, axis = 0)
        print("Missed cloudy images removed")

        x, _, _ = remove_cloud_and_shadows(sentinel2, clouds, shadows, image_dates)
        print("Clouds and shadows interpolated")


        index = 0
        for start_x, end_x in zip(range(0, 633, 126), range(128, 633, 126)):
            for start_y, end_y in zip(range(0, 633, 126), range(128, 633, 126)):
                print(index)
                if not os.path.exists(folder + "processed/{}/{}.npy".format(str(y_vals[index]), str(x_vals[index]))):
                    subtile = x[:, start_x:end_x, start_y:end_y, :]
                    resolved = superresolve(subtile)
                    subtile[:, :, :, 4:10] = resolved
                    dem_i = np.tile(dem[np.newaxis, start_x:end_x, start_y:end_y, :], (x.shape[0], 1, 1, 1))
                    subtile = np.concatenate([subtile, dem_i / 90], axis = -1)
                    subtile, amin = evi(subtile, verbose = True)
                    subtile = bi(subtile, verbose = True)
                    subtile = msavi2(subtile, verbose = True)
                    subtile = si(subtile, verbose = True)

                    subtile = calculate_and_save_best_images(subtile, image_dates)
                    subtile = interpolate_array(subtile)
                    subtile = np.concatenate([subtile, sentinel1[:, start_x:end_x,
                                                                start_y:end_y, :]], axis = -1)


                    out_y_folder = folder + "processed/{}/".format(str(y_vals[index]))
                    if not os.path.exists(os.path.realpath(out_y_folder)):
                        os.makedirs(os.path.realpath(out_y_folder))
                    np.save(folder + "processed/{}/{}.npy".format(str(y_vals[index]), str(x_vals[index])), subtile)
                index += 1
            
def clean_up_folders():
    pass

In [20]:
coord1 = offset_x(coords, 6300)
coord1 = offset_y(coord1 , 6300)
#coord1 = (14.355, 10.694)
calculate_area([coords, coord1])

-70.462961 18.872589
-70.40316756213878 18.873049611040003
-70.462961 18.872589
3969


In [21]:
from tqdm import tnrange, tqdm_notebook
import math
#start at 410
# 745 after CC
for x_tile in range(0, 1):
    for y_tile in range(0, 1):
        print("X: {} Y:{}".format(x_tile, y_tile))
        s1 = download_large_tile(coord = coords, step_x = x_tile, step_y = y_tile)
        process_large_tile(x_tile, y_tile)
        #clean_up_folders(x_tile, y_tile)

X: 0 Y:0
(-70.462961, 18.872589)
-70.462961 18.872589
-70.46305590883179 18.87258825360633
-70.462961 18.872589
-70.4030726505521 18.873050326902963
([-70.40354799691623, 18.930061533071747], [-70.4630551247406, 18.872497905130146])
-70.462961 18.872589
-70.46315081765911 18.872587507164276
-70.462961 18.872589
-70.40297773896116 18.873051042717535
0_0
Making raw folders
Calculating cloud cover
34



Cloud probs: (81, 632, 632)
Loading formatted geocoded file...
NA
SENT_DESC
OOB [798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848.
 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848.
 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848.
 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848. 798848.
 798848.  12202.  14889.   5918. 798848. 798848. 798848. 798848. 798848.
 798848. 798848. 798848. 798848. 798848. 798848. 798848.  10511. 798848.
 798848.  12408. 798848.  10229. 798848. 798848. 798848. 798848.  12947.
 798848.  11680. 798848.   4372.  10932. 798848.  11797. 798848. 798848.
 798848. 798848. 798848. 798848.  11086. 798848. 798848. 798848. 798848.
 798848. 798848. 798848. 798848.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Maximum time distance: 65
Original size: (81, 640, 626, 6)
Original 10 size: (81, 640, 626, 4)
[4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0] [0, 1, 2, 3, 4, 0, 


A total of 91060 pixels were interpolated
Clouds and shadows interpolated
0
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 13s 555ms/step
There are: 0 out of bounds EVI
()
evis error: 0.10265700631953827, 1.2288784350191515, 0 steps, clipping to -1.5, 1.5
Maximum time distance: 50
1
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 15s 615ms/step
There are: 0 out of bounds EVI
()
evis error: 0.012182785367310541, 2.1839703046037338, 0 steps, clipping to -1.5, 1.5
Maximum time distance: 50
2
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 13s 558ms/step
There are: 4 out of bounds EVI
()
evis error: -150.0064077727774, 7.621863495294457, 3 steps, clipping to -1.5, 1.5
Maximum time distance: 50


/Users/john.brandt/Documents/GitHub/restoration-mapper/src/utils-bilinear.py:102: RuntimeWarning: invalid value encountered in power
  sis = np.power( (1-BLUE) * (1 - GREEN) * (1 - RED), 1/3)


3
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 14s 589ms/step
There are: 5 out of bounds EVI
()
evis error: -4.129746345482756, 18.849354315370018, 5 steps, clipping to -1.5, 1.5
Maximum time distance: 50
4
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 14s 579ms/step
There are: 34 out of bounds EVI
()
evis error: -115.58368020027218, 92.35926719411596, 13 steps, clipping to -1.5, 1.5
Maximum time distance: 50
5
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 16s 653ms/step
There are: 1 out of bounds EVI
()
evis error: 0.009705759954959155, 4.047812485296937, 1 steps, clipping to -1.5, 1.5
Maximum time distance: 50
6
Predicting using file: ../src/dsen2/models/s2_032_lr_1e-04.hdf5
24/24 [==============================] - 15s 622ms/step
There are: 4 out of bounds EVI
()
evis error: -34.740409028981915, 4.04781248529